In [1]:
import cv2

tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW','MOSSE', 'CSRT']
tracker_type = tracker_types[-1]

if tracker_type == 'BOOSTING':
    tracker = cv2.legacy.TrackerBoosting_create()
if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create() 
if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create() 
if tracker_type == 'TLD':
    tracker = cv2.legacy.TrackerTLD_create() 
if tracker_type == 'MEDIANFLOW':
    tracker = cv2.legacy.TrackerMedianFlow_create() 
if tracker_type == 'MOSSE':
    tracker = cv2.legacy.TrackerMOSSE_create()
if tracker_type == "CSRT":
    tracker = cv2.TrackerCSRT_create()

In [2]:
# Get the video file and read it
video = cv2.VideoCapture("race.mp4")
ret, frame = video.read()

frame_height, frame_width = frame.shape[:2]
# Resize the video for a more convenient view
frame = cv2.resize(frame, (frame_width // 2, frame_height // 2))
# Initialize video writer to save the results
output = cv2.VideoWriter(f'{tracker_type}.avi',
                         cv2.VideoWriter_fourcc(*'XVID'), 60.0,
                         (frame_width // 2, frame_height // 2), True)

if not ret:
    print('Cannot read the video')

# Select the bounding boxes for two people in the first frame
print("Select the first person's bounding box:")
bbox1 = cv2.selectROI(frame, False)
print("Select the second person's bounding box:")
bbox2 = cv2.selectROI(frame, False)

# Initialize two separate trackers for the two bounding boxes
tracker1 = cv2.TrackerCSRT_create()
tracker2 = cv2.TrackerCSRT_create()

# Initialize the trackers with the selected bounding boxes
tracker1.init(frame, bbox1)
tracker2.init(frame, bbox2)

# Start tracking
while True:
    ret, frame = video.read()
    frame = cv2.resize(frame, (frame_width // 2, frame_height // 2))

    if not ret:
        print('Something went wrong')
        break

    timer = cv2.getTickCount()

    # Update the tracked bounding boxes for both people
    ret1, bbox1 = tracker1.update(frame)
    ret2, bbox2 = tracker2.update(frame)

    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

    if ret1:
        p1 = (int(bbox1[0]), int(bbox1[1]))
        p2 = (int(bbox1[0] + bbox1[2]), int(bbox1[1] + bbox1[3]))
        cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)

    if ret2:
        p1 = (int(bbox2[0]), int(bbox2[1]))
        p2 = (int(bbox2[0] + bbox2[2]), int(bbox2[1] + bbox2[3]))
        cv2.rectangle(frame, p1, p2, (0, 255, 0), 2, 1)

    cv2.putText(frame, tracker_type + " Tracker", (100, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 2)
    cv2.putText(frame, "FPS : " + str(int(fps)), (100, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 2)

    cv2.imshow("Tracking", frame)
    output.write(frame)

    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break

video.release()
output.release()
cv2.destroyAllWindows()


Select the first person's bounding box:
Select the second person's bounding box:


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
